In [1]:
import boto3
import json

In [2]:
# first we will find out which user or role is being used by sagemaker
# We will assign BedrockFUllAccess to it. 
session = boto3.Session()
sts_client = session.client('sts')
identity = sts_client.get_caller_identity()
print(identity['Arn'])

arn:aws:sts::794038231401:assumed-role/SageMaker-ExecutionRole-20250103T203496/SageMaker


In [3]:
# now we will check if we have access to the model using the specific region us-west-2 (change to us-west-2 in the bedrock console to request access in that region)

In [4]:
bedrock = boto3.client(service_name='bedrock', region_name="us-west-2")
bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name="us-west-2")

In [5]:
# bedrock.list_foundation_models()
# print the list of foundation models just using the name
for model in bedrock.list_foundation_models()['modelSummaries']:
    print(model['modelId'])

amazon.titan-tg1-large
amazon.titan-embed-g1-text-02
amazon.titan-text-lite-v1:0:4k
amazon.titan-text-lite-v1
amazon.titan-text-express-v1:0:8k
amazon.titan-text-express-v1
amazon.nova-pro-v1:0
amazon.nova-lite-v1:0
amazon.nova-micro-v1:0
amazon.titan-embed-text-v1:2:8k
amazon.titan-embed-text-v1
amazon.titan-embed-text-v2:0
amazon.titan-embed-image-v1:0
amazon.titan-embed-image-v1
amazon.titan-image-generator-v1:0
amazon.titan-image-generator-v1
amazon.titan-image-generator-v2:0
amazon.rerank-v1:0
stability.stable-diffusion-xl-v1:0
stability.stable-diffusion-xl-v1
stability.sd3-large-v1:0
stability.sd3-5-large-v1:0
stability.stable-image-core-v1:0
stability.stable-image-core-v1:1
stability.stable-image-ultra-v1:0
stability.stable-image-ultra-v1:1
anthropic.claude-3-5-sonnet-20241022-v2:0:18k
anthropic.claude-3-5-sonnet-20241022-v2:0:51k
anthropic.claude-3-5-sonnet-20241022-v2:0:200k
anthropic.claude-3-5-sonnet-20241022-v2:0
anthropic.claude-3-5-haiku-20241022-v1:0
anthropic.claude-ins

In [6]:
# test the model
korean_text = """
한국은 동아시아에 위치한 나라로, 풍부한 역사와 문화를 자랑합니다. 고대로부터 현대에 이르기까지 한국은 수많은 변화를 겪어왔습니다. 삼국시대, 고려시대, 조선시대를 거쳐 현대의 대한민국에 이르기까지 한국인들은 독특한 문화와 전통을 발전시켜 왔습니다.

한국의 문화는 유교, 불교, 샤머니즘 등 다양한 철학과 종교의 영향을 받았습니다. 이러한 영향은 한국의 예술, 건축, 음식, 그리고 일상생활의 여러 측면에서 찾아볼 수 있습니다. 특히 한글이라는 독창적인 문자 체계는 한국 문화의 자부심 중 하나입니다.

현대 한국은 급속한 경제 성장과 기술 혁신으로 세계적인 주목을 받고 있습니다. 삼성, 현대, LG와 같은 글로벌 기업들이 한국 경제의 중추적 역할을 하고 있으며, K-pop과 한국 드라마로 대표되는 한류는 전 세계적으로 큰 인기를 얻고 있습니다.

그러나 한국 사회는 여전히 많은 도전에 직면해 있습니다. 저출산 고령화, 청년 실업, 사회 불평등 등의 문제는 한국이 해결해야 할 중요한 과제입니다. 또한 남북 관계와 주변국과의 외교 관계도 한국의 미래에 중요한 영향을 미칠 것입니다.

한국인들은 이러한 도전을 극복하고 더 나은 미래를 만들어가기 위해 노력하고 있습니다. 교육에 대한 높은 열정, 근면성실한 국민성, 그리고 혁신을 추구하는 정신은 한국이 앞으로도 계속 발전해 나갈 수 있게 하는 원동력이 될 것입니다.
"""

prompt = f"Translate and summarize the following korean text to english: {korean_text}. Make it 1/5 half the size"

In [7]:

body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 500,
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ]
})

response = bedrock_runtime.invoke_model(
    modelId='anthropic.claude-3-5-sonnet-20240620-v1:0',  # Replace with the appropriate model ID
    body=body
)

print(response)

{'ResponseMetadata': {'RequestId': '714e5a0d-e240-4c65-826f-e62229e9d4dd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 08 Jan 2025 14:10:48 GMT', 'content-type': 'application/json', 'content-length': '1184', 'connection': 'keep-alive', 'x-amzn-requestid': '714e5a0d-e240-4c65-826f-e62229e9d4dd', 'x-amzn-bedrock-invocation-latency': '5200', 'x-amzn-bedrock-output-token-count': '187', 'x-amzn-bedrock-input-token-count': '731'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x7f8be7e0c490>}


In [8]:
# Read and parse the StreamingBody
response_body = json.loads(response['body'].read().decode('utf-8'))

print("\nModel's response:")
print(response_body['content'][0]['text'])


Model's response:
Here's a summarized English translation, reduced to approximately one-fifth of the original length:

Korea, located in East Asia, boasts a rich history and culture. From ancient times to the present, Korea has undergone numerous changes, developing unique traditions through various historical periods. Korean culture is influenced by Confucianism, Buddhism, and shamanism, evident in its art, architecture, and daily life. The Korean alphabet, Hangul, is a source of cultural pride.

Modern Korea has gained global attention for its rapid economic growth and technological innovation, with companies like Samsung and Hyundai playing crucial roles. K-pop and Korean dramas have become internationally popular. However, Korea faces challenges such as an aging population, youth unemployment, and social inequality. Koreans are working to overcome these issues, driven by their passion for education, diligence, and innovative spirit.
